In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:

import kagglehub

# Download latest version
path = kagglehub.dataset_download("alessandrasala79/ai-vs-human-generated-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/alessandrasala79/ai-vs-human-generated-dataset/versions/4


In [29]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from IPython.display import display
import seaborn as sns
import time
import random

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from skimage.feature import hog
from skimage.feature import local_binary_pattern

In [32]:
def set_seed(seed=42):
    np.random.seed(seed)  # For NumPy
    random.seed(seed)  # For Python's random module

set_seed()

In [33]:

TRAIN_CSV = path + '/train.csv'
TEST_CSV = path + '/test.csv'
DATA_DIR = path

In [34]:
train_df = pd.read_csv(TRAIN_CSV)
train_df = train_df[['file_name','label']]
train_df.columns = ['id','label']

In [35]:
test_df = pd.read_csv(TEST_CSV)
test_df.head()

,id
0,test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg
1,test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg
2,test_data_v2/eb364dd2dfe34feda0e52466b7ce7956.jpg
3,test_data_v2/f76c2580e9644d85a741a42c6f6b39c0.jpg
4,test_data_v2/a16495c578b7494683805484ca27cf9f.jpg


In [36]:
IMG_SIZE = (150,150)

In [37]:
train_size = len(train_df) // 2

In [38]:
def extract_lbp_features(img_path, radius=3, points=24,method="uniform"):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Đọc ảnh xám
    if img is None:
        print(f"Error loading image: {img_path}")
        return np.zeros(points + 2)  # Tránh lỗi khi ảnh không load được

    img = cv2.resize(img, (64, 64))  # Resize ảnh về kích thước cố định
    lbp = local_binary_pattern(img, points, radius, method)  # Áp dụng LBP

    # Tính histogram của các giá trị LBP
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, points + 3), range=(0, points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)  # Chuẩn hóa histogram tránh lỗi chia cho 0
    return hist


In [39]:
X = np.array([extract_lbp_features(os.path.join(path,img_path)) for img_path in train_df['id']])
y = np.array(train_df['label'])

KeyboardInterrupt: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:

# Defining the parameters grid for GridSearchCV
param_grid={'C':[0.1,1,10,100],
            'gamma':[0.0001,0.001,0.1,1],
            'kernel':['rbf','poly']}

# Creating a support vector classifier
svc=svm.SVC(probability=True)

# Creating a model using GridSearchCV with the parameters grid
model=GridSearchCV(svc,param_grid)


In [ ]:
model.fit(X_train,y_train)

In [ ]:
import joblib

# save
save_path = "/content/drive/My Drive/models/model.pkl"
joblib.dump(model,save_path)